In [1]:
import os 
import sys
from importlib import reload

# Import Pytorch & Huggingface ==========================
#set cache dir before imports 
from configuration import huggingface_path as cache_dir
#cache_dir = cache_path_here
os.environ['HF_HOME'] = f"{cache_dir}/huggingface"
os.environ['TRANSFORMERS_CACHE']= f"{cache_dir}/huggingface/hub"
os.environ['TORCH_HOME']= f"{cache_dir}/torch"
os.environ['XDG_CACHE_HOME']= cache_dir

#imports 
import torch
from torch.cuda import memory_allocated, empty_cache
from datasets import load_dataset
from torch.utils.data import DataLoader

# Import local functions ==================================
import activation_extractor
from activation_extractor import *
reload(activation_extractor)

/orfeo/cephfs/scratch/area/evillegas/glm/dgxtorch/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


<module 'activation_extractor' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/__init__.py'>

In [2]:
from activation_extractor.extractors import intermediateExtractorBase 
reload(intermediateExtractorBase)
from activation_extractor.inferencers import inferencerBase
reload(inferencerBase)
from activation_extractor.model_functions import load_models, tokenize_funs, inference_funs, default_hooked_layers
reload(load_models)
reload(tokenize_funs)
reload(inference_funs)
reload(default_hooked_layers)

<module 'activation_extractor.model_functions.default_hooked_layers' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/model_functions/default_hooked_layers.py'>

In [3]:
#install editable version of this repository
#!python -m pip install -e .

# Proteins/DNA

In [12]:
#load data
sequences = ["AAAAAAAAAAA", "HHHHHHHHHHHHHH"]

In [13]:
model_name = "facebook/esm2_t6_8M_UR50D"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

In [14]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(sequences) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [15]:
extractor.get_outputs()

{'esm.embeddings': tensor([[[-0.1720, -0.2421, -0.1315,  ...,  0.1121, -0.0918, -0.0569],
          [-0.0587, -0.0105,  0.0076,  ...,  0.0129, -0.0387, -0.1703],
          [-0.0587, -0.0105,  0.0076,  ...,  0.0129, -0.0387, -0.1703],
          ...,
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000]],
 
         [[-0.1720, -0.2421, -0.1315,  ...,  0.1121, -0.0918, -0.0569],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          ...,
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [ 0.0643, -0.2711, -0.1030,  ..., -0.1941, -0.3021, -0.0014]]],
        device='cuda:0', grad_fn=<MulBackward0>),
 'esm.encoder.layer.0.output': te

# Images 

In [16]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [17]:
model_name = "google/vit-base-patch16-224-in21k"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

Some weights of ViTForMaskedImageModeling were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['decoder.0.bias', 'decoder.0.weight', 'embeddings.mask_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/orfeo/cephfs/scratch/area/evillegas/glm/dgxtorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
#intermediate activation extractor
layers_to_hook = default_hooked_layers.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = intermediateExtractorBase.IntermediateExtractorBase(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(image)
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [19]:
extractor.get_outputs()

{'vit.encoder.layer.0': (tensor([[[-0.2115, -0.0856, -0.7374,  ...,  0.4718, -0.2834, -0.8569],
           [ 0.5827,  0.1099, -0.3696,  ...,  0.5110, -1.1132, -2.2483],
           [ 0.1236, -0.1317,  0.0076,  ..., -0.2055, -0.4953, -2.4256],
           ...,
           [-0.4928,  0.3414, -0.4098,  ..., -0.3559,  0.2219, -1.4780],
           [-0.3385, -0.1141, -0.1852,  ..., -0.0454,  0.2772, -2.1655],
           [ 0.4095, -1.3829, -0.3933,  ..., -0.2326, -0.3587, -1.6958]]],
         device='cuda:0', grad_fn=<AddBackward0>),
  None),
 'vit.encoder.layer.1': (tensor([[[-0.0784, -0.1944, -0.6273,  ...,  0.4168, -0.5869, -0.4801],
           [ 1.5815,  1.5037,  0.3247,  ...,  0.6719, -1.0244, -3.1346],
           [ 0.5253,  1.2845,  0.8367,  ...,  0.3153, -0.4834, -3.2324],
           ...,
           [ 0.0944,  0.8752,  0.2000,  ...,  0.1214, -0.3376, -2.1262],
           [-0.2992,  0.4218, -0.2166,  ..., -0.1204,  0.8697, -3.5605],
           [ 1.3042,  0.3457,  0.3472,  ..., -1.3195, -0.

# Multimodal

In [28]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = ["a photo of a cat", "a photo of a dog"]

input_data = {"text":text,
             "image":image}

In [22]:
model_name = "openai/clip-vit-base-patch32"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

/orfeo/cephfs/scratch/area/evillegas/glm/dgxtorch/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [29]:
#intermediate activation extractor
layers_to_hook = default_hooked_layers.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = intermediateExtractorBase.IntermediateExtractorBase(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(input_data)
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [30]:
extractor.get_outputs()

{'vision_model.embeddings': tensor([[[ 0.0026,  0.0012, -0.0073,  ..., -0.0050,  0.0185, -0.0064],
          [ 0.1555,  0.4286,  0.1771,  ...,  0.1970,  0.3349,  0.3738],
          [ 0.2190,  0.3191,  0.1995,  ...,  0.1502,  0.2139,  0.1767],
          ...,
          [ 0.2894,  0.1639,  0.3091,  ...,  0.1865,  0.4500,  0.3453],
          [ 0.3051,  0.1528,  0.2244,  ...,  0.1944,  0.3254,  0.3451],
          [ 0.3621,  0.0782,  0.2873,  ...,  0.1406,  0.4511,  0.2907]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 'vision_model.encoder.layers.0': (tensor([[[-0.0175, -0.0070,  0.0051,  ...,  0.0029, -0.0019, -0.0091],
           [ 0.0035,  0.1063, -0.0019,  ...,  0.0285,  0.1156,  0.0895],
           [ 0.0495,  0.0028,  0.0183,  ..., -0.0497,  0.0761, -0.0045],
           ...,
           [ 0.0188, -0.0086,  0.0968,  ..., -0.0140,  0.0856,  0.0124],
           [ 0.0462,  0.0011,  0.0492,  ..., -0.0127,  0.0491, -0.0023],
           [ 0.0570, -0.0292,  0.0805,  ..., -0.0267,  0.077